In [ ]:
from numpy import * 
from pylab import *
from scipy.optimize import curve_fit
import scipy.sparse  
from scipy.sparse import diags 
from scipy.integrate import odeint 
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit # to fit the data in some curve
from itertools import cycle # need in for loop to plot with different dotted lines
import time 
start_time = time.time()
D = 0.01 #0.0366 # mm^2/minute 0.01
beta = 0.564# mm/minute#
H = 2.5 # Henry's constant (dimensionless)
#### parameter based on experimental
S_0 = 0.01 #initial penetration height
x_ref = 10 ##characterstic length x_ref
####
a_0 = 50
m_0 = 0.5
m_ref = m_0
### constraints paramter 
b = 10
#dimensionless number
A_0 = a_0*m_ref*x_ref/D
print('A_0=', A_0)
Bi = beta*x_ref/D
print('Biot=', Bi)
#######
h_0 = S_0/x_ref
####time #####
T_f = 31 #final time in dimension form (minutes)

t_ref = x_ref**2/D ## time scale

Tmax= 0.0031
dt_f = 5e-9
T= int(ceil(Tmax/dt_f))
print('T=', T)

#T = int(T_f/deltaT) # total number of nodes in time step
T_exp = [0, 1, 2, 3]# interested to see the plots in these time (minutes)
T_dim = [round(number/(t_ref*dt_f)) for number in T_exp] # time slice to plot the experiment 
#T_dim = [round(number/(dt_f)) for number in [0,Tmax/4,Tmax/2, 3*Tmax/4]] 
print(T_dim)
###space discretization in dimensionless form
N = 101
x = linspace(0,1,N) 
h = 1/(N -1) 

##### tridiagonal matrix for M time derivative 
M = diags([h/6, 2*h/3, h/6], [-1, 0, 1], shape=(N, N)).toarray() 
M[0,0] = M[0, 0]/2 
M[N-1, N-1] =  M[N-1, N-1]/2 
#print('M =', M) 
Min = linalg.inv(M) 

#######tridiagonal matrix for K (grad(u).grad(phi)) 
K = diags([-1/h, 2/h, -1/h], [-1, 0, 1], shape=(N, N)).toarray() 
K[0,0] = K[0, 0]/2 
K[N-1, N-1] =  K[N-1, N-1]/2 

######## tridigonal matrix for mixed term  
diagonals = zeros((3, N))   # 3 diagonals 
for i in range(1, N-1): 
    diagonals[1,i] = -h/3  
diagonals[1, 0] = -1/6*(h +3*x[0]) #h/3 - x[1]/2  
diagonals[1, N-1] =  1/6*(3*x[N-1]-h) #h/3 - x[N-2]/2     
for i in range(N-1):     
    diagonals[2,i+1] = x[i+1]/2 - h/3 
for i in range(N-1):     
    diagonals[0,i] = -(x[i]/2 + h/3)   
k = array([diagonals[0,0:N-1], diagonals[1], diagonals[2,1:]], dtype=object) 
A = diags(k,[-1,0,1]).toarray() 

#vectors e_1 and e_{N} (it is needed for boundary terms)
E1 = N*[0] 
E1[0] = 1
EN = N*[0]
EN[N-1] = 1

In = [0.5]

Sigma_t = []
MV_BC = []
for alpha in In:
   
    def Model(u, t): 
        dwdt = A_0*(u[N-1] - alpha/x_ref*u[N]/m_ref) 
        dudt = (dwdt/u[N])*dot(dot(Min, A), u[0:N]) - (1/(u[N])**2)*dot(dot(Min, K), u[0:N]) + (1/u[N])*Bi*(b/m_ref - H*u[0])*dot(Min,E1) - (dwdt/u[N])*u[N-1]*dot(Min,EN)   
        dudt = list(dudt) 
        dudt.append(dwdt) 
        return dudt
    u0 = (N)*[m_0/m_ref] 
    u0.append(h_0) 
    #print(u0) 
    #time discretization
    t = linspace(0, Tmax, T) 
     #solve ODE 
    z = odeint(Model, u0, t) 
    m = z[:,:]
    print(m[:,0])
    hh=z[:,-1]# extracting the data of moving boundary
     ######trasnfering back to moving doamin 
    md = np.multiply(x,np.transpose([hh])) #moving domain over time 
    mm = np.multiply(m[:,:-1],np.transpose([hh])) #need for computing the mass
    plt.figure()
    
    #plt.plot(md[T_dim[0], :], m[T_dim[0], :-1], linewidth=2) 
    plt.plot(md[T_dim[1], :], m[T_dim[1], :-1], linewidth=2) 
    plt.plot(md[T_dim[2], :], m[T_dim[2], :-1], linewidth=2) 
    plt.plot(md[T_dim[3], :], m[T_dim[3], :-1], linewidth=2) 
    plt.grid()
    plt.show()
    #x_0 = h0*x_c
    
    #ploting in dimensional variable
    t_1 = t*t_ref #time nodes in moving domain t = tau*t_re
    St = x_ref*hh #z[:,N] # the position of moving boundary s(t) = x_ref*h(tau)
    
    
    plt.figure()
    plt.plot(t, hh, linewidth=2) 
    plt.grid()
    plt.show()
    
    ########### ploting in the dimensional form ######
    md_d = md*x_ref #moving lenght (domain) in dimensional form)
    plt.figure()
    plt.plot(md_d[T_dim[0],:], m_ref*m[T_dim[0],:-1], linewidth=2)
    plt.plot(md_d[T_dim[1],:],  m_ref*m[T_dim[1],:-1], linewidth=2)
    plt.plot(md_d[T_dim[2],:],  m_ref*m[T_dim[2],:-1], linewidth=2)
    plt.plot(md_d[T_dim[3],:],  m_ref*m[T_dim[3],:-1], linewidth=2)
    plt.legend([r"$t= 0$ min", r"$t = 3.5$ min", r"$t= 10$ min",\
                r"$t = 30$ min"])
    plt.grid()
    plt.show()
    ##########
    plt.figure()
    plt.plot(t_1, St, linewidth=2) 
    plt.plot([0, 3.5, 10, 30], [0, 1, 2, 2], "o", linewidth=5)
    plt.legend([r"FEM", r"Experiment data"], loc='upper left') 
    plt.grid()
    plt.show()
   
    ####calculating the number of walker over time 
    Total_mass_over_time = m_ref*mm[:, :].sum(axis=1)
    print('total_walker in time=',Total_mass_over_time[: -1])
    plt.figure()
    plt.plot(t_1, h*Total_mass_over_time[:])
    plt.xlabel(r'Time, [min]')
    plt.ylabel(r'Total mass of concentration')
    plt.show()
end_time = time.time()
print('computation time=', (end_time-start_time)/60, 'minute(s)') 

In [ ]:
#RWM for my problem dimensionlal form Lab experiment paper PEM  Section 4.3  2023-09-22
#%matplotlib inline
from math import ceil, floor, erf
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import random
import numpy as np
import time
start_time = time.time()
L=2 # Length of domain
########## Constants for my problem
D = 0.01 #0.00366 # diffusion in minutes ( mm^2/ minutes)
d = 1
beta = 0.564# mm/minutes#
H = 2.5
####  parameter based on experimental
S_0 = 0.01
x_ref = 10
####
a_0 = 5
m_0 = 0.5
m_ref = m_0
### constraints paramter 
########
T_f = 500
t_ref = x_ref**2/D
b = 10
#dimensionless number
A_0 = a_0*m_ref*x_ref/D
Bi = beta*x_ref/D
########## 
t_max = 0.0031
#t_max= 0.011/3 # Maximum time
nn= [10, 50]# Number of walkers.
#dt = 5e-08
K = 0.5 # this constant we define for sigma, for example, sigma(h(tau)) = 5*h(tau)
no_real = 1 # no of realization
time_refine = 2
#Dt = [5e-08]#, 5e-08/time_refine]
dt = 5e-08
TT = []
SS = []

#for dt in Dt:
for n in nn:    
    start_time = time.time()
    List_T = []
    List_S = []
    for j in range(no_real):
        dx = np.sqrt(2*d*dt)
        NN = int(dt/dt_f) # number to filter the data from finite element data
        N_x=ceil(L/dx)
        N_t=ceil(t_max/dt) # Number of points in the spatial- and time-domains.
        T=np.zeros((N_x, N_t), dtype=int)# Matrix representing T(x,t), initially set to 0.
        s_vector=np.zeros(N_t)# Vector representing the moving boundary s(t).
        ds = np.zeros(N_t)
        ds[0] = dt*A_0*(1 - K*S_0/m_0)
        s= S_0/x_ref # initial value
        j_t=0
        j_s=floor(s/dx) # Indices for time and the position of the moving boundary s(t).
        print('J_s=', j_s)
        n_of_hit = 0
        n_of_non_hit =0
        pos = 0
        while j_t < N_t-1 and j_s < N_x-1:
            if j_t == 0: 
                for j in range(j_s):
                    T[j, 0] = 1*n
            else: 
                T[0, j_t] = round(((n*dx*Bi*b/(m_ref)+ T[1,j_t])/(1+Bi*H*dx))) #robins bc
            s_vector[j_t] = s
            
            for j_x in range(N_x):
                    #print('j_x =', j_x)
                    if T[j_x, j_t] < 0:
                        sign = -1
                    else:
                        sign = 1
                    for j in range(sign*T[j_x, j_t]):
                        p= 2*round(random.uniform(0, 1))-1 # =+-1, with P(+1)=P(-1)=1/2.
                        if j_x+p > 0 and j_x + p <= j_s  and j_x <= N_x-1: # A walker move if it has not reached the boundaries.
                            T[j_x +p, j_t+1] = T[j_x+p, j_t+1] + 1*sign
                        elif j_x + p == j_s+1: 
                            ds[j_t] = dt*A_0*(T[j_s, j_t] - (K/x_ref)*s/(m_ref)) 
                            P = (1/n)*np.sqrt(2/np.pi)*A_0*(T[j_s, j_t]*sign - (K/x_ref)*s/(m_ref))*np.sqrt(np.pi*dt) 
                            if  P<=0:
                                print('P is negative')
                            if P >1:
                                print('P is greater than 1 in j_x and j_t', j_x, j_t)
                            r = random.uniform(0,1) 
                            if T[j_s, j_t] - (K/x_ref)*s/(m_ref)  < 0:
                                #print('oh it gives negative speed', j_s, j_x, j_t, T[j_s, j_t] )
                                pos = pos +1
                            if r<P:
                                T[j_s , j_t+1] = T[j_s, j_t+1] +1
                                s = s + ds[j_t]/n*sign #
                                n_of_hit = n_of_hit +1
                            else:
                                T[j_s-1, j_t+1] = T[j_s-1, j_t+1] + 1*sign 
                                n_of_non_hit = n_of_non_hit +1
                            j_s =   floor(s/dx)

            j_t = j_t +1
        print('n_of_hit=', n_of_hit)
        print('n_of_non_hit=', n_of_non_hit)
        print('pos =', pos)
        T = T/(n*no_real)
        List_T.append(T)
        List_S.append(s_vector/no_real)
    T = np.sum(List_T, 0)  
    s_vector = np.sum(List_S, 0) 
    
    #ploting concentration profile in 3D
    x = np.linspace(0, (N_x-1)*dx, N_x) #space grid points
    y = np.linspace(0, (N_t-1)*dt, N_t) #time grid points
    X, Y = np.meshgrid(x, y)
    mappable = plt.cm.ScalarMappable(cmap=plt.cm.viridis)
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_surface(X.transpose(), Y.transpose(), T, cmap=mappable.cmap)
    plt.colorbar(mappable)
    t_vector = y[:-1]
    ax.plot3D(s_vector[0:j_t], t_vector, 0*t_vector, 'ro') 
    
    S_t = np.load('MovingBc.npy') #loading the data for moving boundary that we saved in FEM code
    S_T = St[::NN]
    fig = plt.figure()
    plt.plot(t_vector, s_vector[0:j_t], 'r-')
    plt.xlabel('t')
    plt.ylabel('s(t)')
    plt.plot()
    
    ####ploting the concentration profile in non-dimensional variable 
    fig = plt.figure()
    plt.plot(x, T[:, -3], 'r-')
    plt.xlabel('z')
    plt.ylabel(r'u($\tau$, z)')
    plt.plot()
    
    ####ploting the concentration profile in dimensional variable
    fig = plt.figure(dpi=500)
    plt.plot(x*x_ref, m_ref*T[:, int((1/NN)*T_dim[2])], 'r-')
    plt.plot(md_d[T_dim[2], :], m_ref*m[T_dim[2],:-1], 'g--', linewidth=2)
    fontsize = 15
    fontweight = 'normal'
    fontproperties = {'weight' : fontweight, 'size' : fontsize}
    plt.legend([r"RWM", r"FEM"], loc='upper right', frameon=False) # loc='upper left', frameon=False
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    #plt.grid()
    plt.xlabel('$z$', fontsize= fontsize, fontweight=fontweight)
    plt.ylabel(r'$u(\tau, z)$', fontsize= fontsize, fontweight=fontweight)
    plt.plot()
    plt.show()

    #ploting in dimensional form and compare the result with FE apprximation, expermiental data
    t_d = t_vector*(x_ref**2/D)
    s_vector_d = s_vector[0:j_t]*x_ref
    plt.figure(dpi=500)
    plt.plot([0, 3.5, 10, 30], [0, 1, 2, 2], 'b*') #lab experimental data
    plt.plot(t_d[:], s_vector_d[:], 'r-') # random walk solution
    plt.plot(t_d[:], S_T[:len(t_d)], 'g--') #finite element solution
    fontsize = 15
    fontweight = 'normal'
    fontproperties = {'weight' : fontweight, 'size' : fontsize}
    plt.legend([r"RWM", r"FEM"], loc='upper left', frameon=False) # loc='upper left', frameon=False
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.xlabel(r'$\tau$', fontsize= fontsize, fontweight=fontweight)
    plt.ylabel(r'$h(\tau)$', fontsize= fontsize, fontweight=fontweight)
    plt.plot()
    
    #### ploting the concentration profile on dimesional form
    T_1 = T*m_ref
    xx =  x_ref*np.linspace(0, (N_x-1)*dx, N_x) #space grid points
    yy = (t_ref)*np.linspace(0, (N_t-1)*dt, N_t) #time grid points
    XX, YY = np.meshgrid(xx, yy)
    mappable = plt.cm.ScalarMappable(cmap=plt.cm.viridis)
    mappable.set_array(T_1)
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot_surface(XX.transpose(), YY.transpose(), T_1, cmap=mappable.cmap, norm=mappable.norm, )
    plt.colorbar(mappable)
    t_vector_1 = yy[:-1]
    print('len_t_vector=', len(t_vector))
    ax.plot3D(x_ref*s_vector[0:j_t], t_vector_1, 0*t_vector_1, 'ro')  
    ax.set_xlabel(r'Penetration region [mm]', rotation=150)
    ax.set_ylabel(r'Time [min]')
    plt.show()
    
    ####calculating the number of walker over time int_u(t, x)dx 
    Total_walker = T_1.sum(axis=0)
    print('total_walker in time=',dx*Total_walker[: -1])
    plt.figure(dpi=500)
    plt.plot(t_d, dx*Total_walker[: -1], 'r-')
    plt.plot(t_1, h*Total_mass_over_time[:], 'g--')
    fontsize = 15
    fontweight = 'normal'
    fontproperties = {'weight' : fontweight, 'size' : fontsize}
    plt.legend([r"RWM", r"FEM"], loc='upper left', frameon=False ) # loc='upper left', frameon=False
    plt.xlabel('Time, [min]', fontsize= fontsize, fontweight=fontweight)
    plt.ylabel(r'Mass, [g/mm$^3$]', fontsize= fontsize, fontweight=fontweight)
    plt.plot()
    TT.append(T_1)
    SS.append(s_vector_d)
    end_time = time.time()
    print('Computation time for n =', (end_time-start_time)/60, 'minute(s)')
#np.savez('mv_front_lab_dt_5e08_n100and500and1k.npz', SS)
#np.savez('con_profile_lab_dt_5e08_n100and500and1k.npz', TT)

fig = plt.figure(dpi=500)
plt.plot(t_d[:], SS[0], 'r--', linewidth = 2) # random walk solution
plt.plot(t_d[:], SS[1], 'b:', linewidth = 2) # random walk solution
plt.plot(t_d[:], S_T[:len(t_d)], 'g-', linewidth = 2) #finite element solution
fontsize = 15
fontweight = 'normal'
fontproperties = {'weight' : fontweight, 'size' : fontsize}
#plt.axis([-0.0000001, np.max(t_d[:])+0.0000001, 0, np.max(S_T[:len(t_d)])+0.001])
plt.legend([r"RWM $(n=1000)$", r"RWM $(n=1500)$",\
            r"RWM $(n=2000)$",r"FEM"], loc='lower right', frameon=False) # loc='upper left', frameon=False
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.xlabel(r'$\tau$', fontsize= fontsize, fontweight=fontweight)
plt.ylabel(r'$h(\tau)$', fontsize= fontsize, fontweight=fontweight)
plt.show()

####ploting for differnnt values for n in same plot
fig = plt.figure(dpi=500)
plt.plot(x*x_ref, TT[0][:, int((1/NN)*T_dim[3])], 'r-', linewidth = 2) #random walk solution
plt.plot(md_d[T_dim[3], :], m_ref*m[T_dim[3],:-1], 'g--' ,linewidth=2) #FEM
fontsize = 15
fontweight = 'normal'
fontproperties = {'weight' : fontweight, 'size' : fontsize}
plt.legend([r"RWM (n=1000)",r"FEM"], loc='upper right', frameon=False) # loc='upper left', frameon=False
plt.xlabel('Penetration region, [mm]', fontsize= fontsize, fontweight=fontweight)
plt.ylabel(r'Concentration profile, [g/mm$^3$]', fontsize= fontsize, fontweight=fontweight)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.plot()
plt.show()

#####plot the left boundary
fig = plt.figure(dpi=500)
plt.plot(t_d, TT[0][0,:-1], 'r-',  linewidth=2 )
plt.plot(t_1, m_ref*m[:,0], 'g--' ,linewidth=2 )
fontsize = 15
fontweight = 'normal'
fontproperties = {'weight' : fontweight, 'size' : fontsize}
plt.legend([r"RWM (n=1000)", r"FEM"], loc='lower right', frameon=False)
plt.xlabel(r'$\tau$', fontsize= fontsize, fontweight=fontweight)
plt.ylabel(r'$u(\tau, 0)$', fontsize= fontsize, fontweight=fontweight)
plt.xticks(fontsize=12)
plt.yticks(fontsize=12)
plt.show()
end_time = time.time()
print('Computation time =', (end_time-start_time)/60, 'minute(s)')
